In [22]:
!nvidia-smi

Tue Apr 15 04:55:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             27W /   70W |   14958MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [23]:
!pip install transformers
!pip install langchain
!pip install chromadb
!pip install pypdf
!pip install xformers
!pip install InstructorEmbedding
!pip install pdf2image

In [24]:
!pip install torch

In [26]:
import torch
print(torch.__version__)


2.6.0+cu124


In [25]:
!pip install auto_gptq

In [27]:
!pip install langchain_community

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [28]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [29]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


In [30]:
!gdown 1mmA_ReCmsJiujVWLOkcxKRDQhp9z6DVE -O pdfs/Film-Contracts.pdf
!gdown 1ckb5NUrg3d3O35-U1kluS59LGMhMqVkt -O pdfs/The-Business-of-Film_3rd.pdf


Downloading...
From: https://drive.google.com/uc?id=1mmA_ReCmsJiujVWLOkcxKRDQhp9z6DVE
To: /content/pdfs/Film-Contracts.pdf
100% 33.6M/33.6M [00:00<00:00, 101MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1ckb5NUrg3d3O35-U1kluS59LGMhMqVkt
To: /content/pdfs/The-Business-of-Film_3rd.pdf
100% 5.35M/5.35M [00:00<00:00, 17.5MB/s]


In [31]:
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

1043

In [ ]:
for doc in docs:
  print(doc.metadata)

{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs/Film-Contracts.pdf', 'total_pages': 645, 'page': 0, 'page_label': '1'}
{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs/Film-Contracts.pdf', 'total_pages': 645, 'page': 1, 'page_label': '2'}
{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs/Film-Contracts.pdf', 'total_pages': 645, 'page': 2, 'page_label': '3'}
{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs

In [32]:
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="hkunlp/instructor-large")

!pip install langchain-huggingface

from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'device': DEVICE}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs=model_kwargs)


In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=1024,
    chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

3380

In [34]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 3min 28s, sys: 1.05 s, total: 3min 29s
Wall time: 3min 25s


## Llama 2 13B

In [35]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
model_name_or_path = "Qwen/Qwen2.5-3B-Instruct"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
!nvidia-smi

Tue Apr 15 05:04:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             30W /   70W |   14964MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [37]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.

If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [38]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [45]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

Device set to use cuda:0


In [46]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [47]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [48]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [49]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
chain_type="stuff",
    #retriever=db.as_retriever(search_kwargs={"k": 2}),
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat

In [50]:
result = qa_chain( 'The process of transforming a film from a script to a commercial moviegoes through five steps:'  )

[INST] <<SYS>>
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<</SYS>>


newly minted producers were first and foremost businessmen, with a focus 
on profit, not art. From the outset of what became the film studio system, 
business interests were paramount.
The Birth of Distribution
The process of transforming a film from a script to a commercial movie 
goes through five steps:
 1 Development: an idea transformed into a script, the attachment of key 
players (producer, director, stars), creation of a budget; in order to 
seek:

newly minted producers were first and foremost businessmen, with a focus 
on profit, not art. From the outset of what became the film studio system, 
business interests were paramount.
The Birth of Distribution
The process of transforming a film from a script to a commercial movie 
goes through five steps:
 1 Development: an idea transformed into a

KeyboardInterrupt: 

In [ ]:
print(result["source_documents"])

[Document(metadata={'author': 'Stephen R. Greenwald', 'creationdate': '2022-09-09T19:14:57+05:30', 'creator': 'Adobe InDesign 14.0 (Windows)', 'keywords': 'business of media; OTT; crowdfunding; film distribution; media business; media distribution; film finance; media entrepreneurship; AR; VR', 'moddate': '2022-09-14T14:37:46+05:30', 'page': 27, 'page_label': '9', 'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'source': 'pdfs/The-Business-of-Film_3rd.pdf', 'subject': 'This updated third edition introduces readers to the business of film at every stage of the filmmaking lifecycle A practical, hands-on guide exploring streaming, development, financing trends, regional/global/online distribution, shifting business models, exhibition, multi-platform delivery, marketing, VR/AR, accounting, and more.', 'title': 'The Business of Film; A Practical Introduction', 'total_pages': 398}, page_content='newly minted producers were first and foremost businessmen, with a focus \non profit, not

In [51]:
result = qa_chain( 'list the key characteristics of the film industry'  )

and[INST] <<SYS>>
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<</SYS>>


DOI: 10.4324/9781003217480-2
Chapter 2
A Business Overview of Film
The film business has certain characteristics that distinguish it from other 
types of commercial enterprises, and that demand expertise in, and mas -
tery of, the business consequences that flow from these attributes.
Key Characteristics of the Industry
While the technological side of the film industry has changed dramatically 
from the very beginning, certain business characteristics remain constant, 
and patterns have emerged regardless of technology.
These characteristics include:
 • film as intellectual property;
 • managing creative talent;
 • film as a global product;
 • film takes a long time to develop, produce, and market;
 • market structure—oligopoly; with a handful of companies—streamers 
and studios—controlling the in

In [ ]:
print(result["source_documents"])

[Document(metadata={'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'total_pages': 398, 'keywords': 'business of media; OTT; crowdfunding; film distribution; media business; media distribution; film finance; media entrepreneurship; AR; VR', 'title': 'The Business of Film; A Practical Introduction', 'creator': 'Adobe InDesign 14.0 (Windows)', 'source': 'pdfs/The-Business-of-Film_3rd.pdf', 'moddate': '2022-09-14T14:37:46+05:30', 'page': 53, 'page_label': '35', 'author': 'Stephen R. Greenwald', 'creationdate': '2022-09-09T19:14:57+05:30', 'subject': 'This updated third edition introduces readers to the business of film at every stage of the filmmaking lifecycle A practical, hands-on guide exploring streaming, development, financing trends, regional/global/online distribution, shifting business models, exhibition, multi-platform delivery, marketing, VR/AR, accounting, and more.'}, page_content='DOI: 10.4324/9781003217480-2\nChapter 2\nA Business Overview of Film\nThe film business 

In [52]:
result = qa_chain( 'give me a contract template for submission release')



 Here's a simplified version of a contract template for submission release:

---

**Submission Release Agreement**

This Submission Release Agreement ("Agreement") is made and entered into as of [DATE], by and between [SUBMITTER NAME], hereinafter referred to as "You," and [PRODUCTION COMPANY NAME], hereinafter referred to as "PRODUCTION COMPANY."

1. **Summary of Material**
   - You hereby submit to PRODUCTION COMPANY certain material(s), including but not limited to scripts, outlines, treatments, drawings, photographs, taped materials, electronic files, etc. All important features of these materials are summarized on Schedule A attached hereto.

2. **Acknowledgment**
   - You acknowledge that this agreement covers and governs any and all submissions of the aforementioned material from now until such time as a formal written contract is executed between You and PRODUCTION COMPANY.

3. **Confidentiality and Non-Disclosure**
   - You confirm that you have disclosed no other features rel

In [ ]:
print(result["source_documents"][0].page_content)

the formal written contract.
 3 Y ou declare that all important features of your Material, and the par-
ticular items being submitted by you (e.g., script, outline, treatment, 
drawings, photographs, taped materials, electronic files, etc.) are sum-
marized on Schedule A annexed to this form, and you have disclosed 
no other features to PRODUCTION COMPANY .
 4 Y ou acknowledge that this release covers and governs any and all of 
the Material, whether first submitted to PRODUCTION COMPANY 
contemporaneously with, or prior to, or following, the execution of 
this release, and applies also to any submission of the Material made 
to PRODUCTION COMPANY by another source, directly or indi -
rectly, by or through you, representatives, and assigns.
 5 Y ou warrant and represent that (i) Y ou are the sole author of 
the Material and all elements thereof, and (ii) no third party has 
any rights, title, or interest therein or thereto. I agree to indemnify


## References
https://github.com/PanQiWei/AutoGPTQ/
